<a href="https://colab.research.google.com/github/albertofernandezvillan/computer-vision-and-deep-learning-course/blob/main/KMeansClusteringOpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
#Import required packages:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import collections
from google.colab.patches import cv2_imshow

In [43]:
def color_quantization(image, k):
    """Performs color quantization using K-means clustering algorithm"""

    # Transform image into 'data':
    data = np.float32(image).reshape((-1, 3))

    # Define the algorithm termination criteria (the maximum number of iterations and/or the desired accuracy):
    # In this case the maximum number of iterations is set to 20 and epsilon = 1.0
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)

    # Apply K-means clustering algorithm:
    # ret, label, center = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    ret, label, center = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_PP_CENTERS)
    
    # At this point we can make the image with k colors
    # Convert center to uint8:
    center = np.uint8(center)

    # Replace pixel values with their center value:
    result = center[label.flatten()]
    result = result.reshape(image.shape)

    # Build the 'color_distribution' legend.
    # We will use the number of pixels assigned to each center value:
    counter = collections.Counter(label.flatten())

    # Calculate the total number of pixels of the input image:
    total = image.shape[0] * image.shape[1]

    # Assign width and height to the color_distribution image:
    desired_width = image.shape[1]
    # The difference between 'desired_height' and 'desired_height_colors'
    # will be the separation between the images
    desired_height = 70
    desired_height_colors = 50

    # Initialize the color_distribution image:
    color_distribution = np.ones((desired_height, desired_width, 3), dtype="uint8") * 255
    # Initialize start:
    start = 0

    for key, value in counter.items():
        # Calculate the normalized value:
        value_normalized = value / total * desired_width

        # Move end to the right position:
        end = start + value_normalized

        # Draw rectangle corresponding to the current color:
        cv2.rectangle(color_distribution, (int(start), 0), (int(end), desired_height_colors), center[key].tolist(), -1)
        # Update start:
        start = end

    return np.vstack((color_distribution, result))

In [ ]:
!wget "https://raw.githubusercontent.com/albertofernandezvillan/computer-vision-and-deep-learning-course/main/assets/thermal-image.JPG"

In [ ]:
img = cv2.imread("thermal-image.JPG")
cv2_imshow(img)

In [ ]:
# Apply color quantization (K-means with K=2)
img_result = color_quantization(img, 2)
cv2_imshow(img_result)

In [ ]:
# Apply bilateral filter to reduce noise in the input image:
img_bilateral_blur = cv2.bilateralFilter(img, 100, 120, 120)
cv2_imshow(img_bilateral_blur)

In [ ]:
# Apply color quantization (K-means with K=2)
img_result_bilateral_blur = color_quantization(img_bilateral_blur, 2)
cv2_imshow(img_result_bilateral_blur)